In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

In [2]:
import os
os.chdir("C://Users//mahid//OneDrive//Desktop//Datasciencechannel//chatbot project")

In [3]:
os.listdir()

['classes.pkl', 'intents.json', 'words.pkl']

In [4]:
#reading the files
intents = open('intents.json').read()
intents_file = json.loads(intents)

In [47]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mahid\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [5]:
print(intents_file)

{'intents': [{'tag': 'greeting', 'patterns': ['Hi there', 'How are you', 'Is anyone there?', 'Hey', 'Hola', 'Hello', 'Good day'], 'responses': ['Hello, thanks for asking', 'Good to see you again', 'Hi there, how can I help?'], 'context': ['']}, {'tag': 'goodbye', 'patterns': ['Bye', 'See you later', 'Goodbye', 'Nice chatting to you, bye', 'Till next time'], 'responses': ['See you!', 'Have a nice day', 'Bye! Come back again soon.'], 'context': ['']}, {'tag': 'thanks', 'patterns': ['Thanks', 'Thank you', "That's helpful", 'Awesome, thanks', 'Thanks for helping me'], 'responses': ['Happy to help!', 'Any time!', 'My pleasure'], 'context': ['']}, {'tag': 'noanswer', 'patterns': [], 'responses': ["Sorry, can't understand you", 'Please give me more info', 'Not sure I understand'], 'context': ['']}, {'tag': 'options', 'patterns': ['How you could help me?', 'What you can do?', 'What help you provide?', 'How you can be helpful?', 'What support is offered'], 'responses': ['I can guide you through

In [6]:
#preprocessing of data
words = []
classes = []
documents = []
ignore_letters = ['!', '?', ',', '.']

for intent in intents_file["intents"]:
    for pattern in intent["patterns"]:
        #tokenize of word
        word = nltk.word_tokenize(pattern)
        someword = [i.strip() for i in word if i not in ignore_letters]
        words.extend(someword)
        #add documents in the corpus
        documents.append((someword, intent["tag"]))
        if intent["tag"] not in classes:
            classes.append(intent["tag"])
print(classes)
print(documents)

['greeting', 'goodbye', 'thanks', 'options', 'adverse_drug', 'blood_pressure', 'blood_pressure_search', 'pharmacy_search', 'hospital_search']
[(['Hi', 'there'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there'], 'greeting'), (['Hey'], 'greeting'), (['Hola'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Nice', 'chatting', 'to', 'you', 'bye'], 'goodbye'), (['Till', 'next', 'time'], 'goodbye'), (['Thanks'], 'thanks'), (['Thank', 'you'], 'thanks'), (['That', "'s", 'helpful'], 'thanks'), (['Awesome', 'thanks'], 'thanks'), (['Thanks', 'for', 'helping', 'me'], 'thanks'), (['How', 'you', 'could', 'help', 'me'], 'options'), (['What', 'you', 'can', 'do'], 'options'), (['What', 'help', 'you', 'provide'], 'options'), (['How', 'you', 'can', 'be', 'helpful'], 'options'), (['What', 'support', 'is', 'offered'], 'options'), (['How', 'to', 'check', 'Adverse', 'drug'

In [7]:
#checkinhg for stopwords
# nltk.download("stopwords")
from nltk.corpus import stopwords
stop_words = set(stopwords.words("english"))
print(stop_words)

{'wouldn', 'at', 'haven', 'should', 'isn', "couldn't", 'with', 'against', 'into', 'hasn', 're', 'i', 'it', 'ourselves', "didn't", 'while', 'theirs', 'they', 'until', 'these', "needn't", 'once', 'after', "mustn't", 'that', 'itself', 'few', 'on', 'was', 'will', 'and', 'out', "haven't", "you'd", 'more', 'mustn', 'those', 'no', "shouldn't", "she's", 'than', 'she', 'some', 'why', 'own', 'above', 'very', 'doesn', 'a', 'he', 'both', 'didn', 'his', 'of', 'shouldn', 'yours', 'then', 'our', 'were', 's', 'have', 'we', 'where', 'yourselves', 'mightn', 'shan', 'during', 'each', 'him', "don't", 'been', 'all', 'here', "mightn't", 'this', "won't", "that'll", 'before', 'down', "wouldn't", 'up', 'had', 'o', 'or', 'himself', 'the', 'other', 'for', 'm', "it's", 'doing', 'weren', 'when', 'because', "you've", 'most', 'only', 'now', 'herself', 'what', 'which', 'her', 'ours', "hadn't", 'how', 'you', 'by', 'd', 'over', 'do', 'below', 'further', "weren't", 'hers', 've', "you're", "you'll", 'couldn', 'whom', 'ha

In [8]:
#lemmatization of word
# print(words)
# nltk.download('wordnet')
words = [lemmatizer.lemmatize(w.lower()) for w in words]
words = sorted(list(set(words)))
print(words)

["'s", 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


In [9]:
# sort classes
classes = sorted(list(set(classes)))
print(classes)
# # documents = combination between patterns and intents
print (len(documents), "documents")
# # classes = intents
print (len(classes), "classes", classes)
# # words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
47 documents
9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
87 unique lemmatized words ["'s", 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thank

In [108]:
# dumping the words and classes into pickle format
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open("classes.pkl",'wb'))

In [109]:
# formatt of reading & loading pickle file
with open("words.pkl","rb") as file:
    d = pickle.load(file)
print(d)
with open("classes.pkl","rb") as file:
    d1 = pickle.load(file)
print(d1)

["'s", 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


In [43]:
#CREATING TRAINING AND TESTING DATA
training = []
output_empty = [0] * len(classes)
# print(documents)
for doc in documents:
    bag = []
    word_patterns = doc[0]
    word_patterns = [lemmatizer.lemmatize(w.lower()) for w in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
#shuffling the features
random.shuffle(training)
# from keras.preprocessing.sequence import pad_sequences
# pad_data = pad_sequences(training, padding = 'post')
# train_new = np.array(pad_data)
# print(train_new)
train_x = training[0]
print(train_x)

[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1]]


In [ ]:
model = Sequential()
model.add(Dense(128, input_shape(len(train_x[0],), activation = 'relu')))
model.add(Dropout(0.5))
model.add(Dense(64,activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0])), activation = 'softmax')

#RNN diease & prescription predicting for a patient problem using RNN & LSTM networks

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

from tensorflow import keras
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

In [17]:
import pandas as pd
import numpy as np
# from tensorflow.keras.preprocessing.text import tokenizer
# from tensorflow.keras.preprocessing.sequence import pad_sequence
# from tensorflow.keras.util import to_categorical
# from sklearn.preprocessing import LabelEncoder
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import LSTM, Embeddings, Dense, Input
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, LSTM
from sklearn.preprocessing import LabelEncoder
import os

In [18]:
os.chdir("C://Users//mahid//OneDrive//Desktop//LTI Payslips")
os.listdir()

['Dec 2022.pdf',
 'Feb 2023.pdf',
 'Jan 2023.pdf',
 'March 2023.pdf',
 'medical_data.csv',
 'Nov 2022.pdf',
 'oct 2022.pdf',
 'Payslip 2023-2024 Financial Year.pdf',
 'Score Report _ Cibil Dashboard.pdf',
 'Sep 2022.pdf',
 '__APSPDCL__GV.pdf',
 '__APSPDCL__GVKR.pdf']

In [19]:
df = pd.read_csv("medical_data.csv")
df.head()

,Patient_Problem,Disease,Prescription
0,"Constant fatigue and muscle weakness, struggli...",Chronic Fatigue Syndrome,"Cognitive behavioral therapy, graded exercise ..."
1,"Frequent severe migraines, sensitivity to ligh...",Migraine with Aura,"Prescription triptans, avoid triggers like bri..."
2,"Sudden weight gain and feeling cold, especiall...",Hypothyroidism,Levothyroxine to regulate thyroid hormone levels.
3,"High fever, sore throat, and swollen lymph nod...",Mononucleosis,"Rest and hydration, ibuprofen for pain."
4,"Excessive thirst and frequent urination, dry m...",Diabetes Mellitus,Insulin therapy and lifestyle changes.


In [25]:
df["Disease"].unique()

array(['Chronic Fatigue Syndrome', 'Migraine with Aura', 'Hypothyroidism',
       'Mononucleosis', 'Diabetes Mellitus', 'Pericarditis',
       'Tuberculosis', 'Pancreatitis', 'Rheumatoid Arthritis',
       'Herpes Simplex', 'Esophageal Reflux', 'Irritable Bowel Syndrome',
       'Scabies', 'Hepatitis', 'Subarachnoid Hemorrhage', 'Lung Cancer',
       'Hemophilia', 'Food Allergy', 'Urinary Tract Infection',
       'Early Onset Dementia', 'Migraine', 'Asthma', 'Panic Attacks',
       'Type 1 Diabetes', 'Irritable Bowel Syndrome (IBS)', 'Cystic Acne',
       'Insomnia', 'Major Depressive Disorder', 'Myopia and Hyperopia',
       'Diabetes Type 2', 'Gout',
       'Chronic Obstructive Pulmonary Disease (COPD)',
       'Benign Positional Vertigo', 'Hyperhidrosis', 'Pharyngitis',
       'Peripheral Neuropathy', 'Pulmonary Embolism',
       'Migraine and Vertigo', 'Gastrointestinal Infection',
       'Contact Dermatitis', 'Chronic Bronchitis', 'Patellar Tendinitis',
       'Myopia', 'Sciatica'

In [26]:
df["Prescription"].value_counts()

Prescription
Thyroid hormone replacement therapy.                             6
Increase salt intake; wear compression stockings.                3
Surgical removal of the appendix.                                3
Antidepressants, psychotherapy.                                  2
Pain relievers and physical therapy; surgery in severe cases.    2
                                                                ..
Anticoagulants; wearing compression stockings.                   1
Sleep hygiene practices; melatonin supplements.                  1
Prenatal vitamins; regular check-ups.                            1
Topical corticosteroids; moisturizers.                           1
Colonoscopy, potentially surgery, chemotherapy.                  1
Name: count, Length: 388, dtype: int64

In [27]:
df["Patient_Problem"].value_counts()

Patient_Problem
Constant fatigue and muscle weakness, struggling to stay awake.      1
I have severe and persistent abdominal pain.                         1
Experiencing sharp knee pain when walking or climbing stairs.        1
Persistent coughing and shortness of breath, especially at night.    1
I've noticed a spreading rash on my arm that's very itchy.           1
                                                                    ..
Swelling and pain in one leg.                                        1
Difficulty falling asleep at night.                                  1
Nausea and vomiting in the morning.                                  1
Skin rash and itching.                                               1
Blood in stool, along with a change in bowel movement frequency.     1
Name: count, Length: 407, dtype: int64

In [20]:
print(df.shape)
print(df.isnull().sum())
print(df.duplicated().sum())

(407, 3)
Patient_Problem    0
Disease            0
Prescription       0
dtype: int64
0


In [21]:
#tokenizing & sequence of data
tokenizer = Tokenizer(num_words = 5000, oov_token = "<00V>")
tokenizer.fit_on_texts(df['Patient_Problem'])
sequence = tokenizer.texts_to_sequences(df['Patient_Problem'])
print(sequence)

[[37, 71, 2, 111, 164, 368, 16, 369, 370], [24, 11, 268, 112, 16, 72, 2, 269], [26, 34, 133, 2, 9, 62, 18, 4, 3, 35, 2, 42], [93, 63, 53, 38, 2, 73, 165, 166, 9, 84, 371], [46, 74, 2, 24, 59, 36, 75, 372], [43, 270, 40, 7, 20, 167, 27, 373, 193, 29, 374, 194], [54, 34, 21, 2, 28, 168, 9, 375], [11, 50, 7, 271, 16, 3, 51, 94, 32, 195], [95, 44, 2, 96, 4, 3, 64, 18, 4, 3, 196, 2, 197], [134, 85, 272, 135, 3, 75, 2, 376], [377, 14, 52, 198, 136, 199], [169, 97, 50, 76, 2, 378], [55, 86, 18, 22, 28, 30, 273, 379, 17, 3, 23], [137, 5, 3, 23, 2, 77, 274, 200, 2, 380, 71], [26, 11, 138, 30, 201, 275, 276, 139, 31], [25, 60, 30, 78, 381, 382, 28, 168], [24, 170, 383, 171, 2, 384, 202, 140, 385], [11, 86, 2, 56, 33, 386, 277, 203], [387, 388, 389, 390, 113, 114, 391, 204], [14, 98, 87, 115, 2, 172], [13, 12, 9, 205, 392, 2, 393, 141, 278, 16, 279], [15, 24, 57, 112, 16, 72, 2, 269], [47, 8, 25, 60, 2, 142, 280, 18, 22, 28], [26, 58, 5, 173, 116, 9, 281], [394, 34, 21, 2, 37, 74], [95, 7, 4, 35,

In [24]:
# taking max length from sequences and applying pad_sequences
max_length = max(len(x) for x in sequence)
pad_sequence = pad_sequences(sequence, maxlen = max_length, padding = 'post')
print(pad_sequence)

[[ 37  71   2 ...   0   0   0]
 [ 24  11 268 ...   0   0   0]
 [ 26  34 133 ...   0   0   0]
 ...
 [ 15 172  14 ...   0   0   0]
 [ 37   9   5 ...   0   0   0]
 [ 78   4 130 ...   0   0   0]]


In [34]:
# Encoding the labels
label_encoder_disease = LabelEncoder()
label_encoder_prescription = LabelEncoder()

disease_labels = label_encoder_disease.fit_transform(df["Disease"])
prescription_labels = label_encoder_prescription.fit_transform(df["Prescription"])

#converting labels into categorical
disease_labels_categorical = to_categorical(disease_labels)
prescription_labels_categorical = to_categorical(prescription_labels)

print(disease_labels_categorical)
print(prescription_labels_categorical)

# print(max(disease_labels))
# print(max(prescription_labels))

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [37]:
# combing both disease and prescription
out = np.hstack((disease_labels_categorical, prescription_labels_categorical))
print(out)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [38]:
# Model Building
print(label_encoder_disease.classes_)

['Achilles Tendinitis' 'Acid Reflux' 'Age-related Macular Degeneration'
 'Allergic Rhinitis' 'Alopecia Areata' "Alzheimer's Disease" 'Anaphylaxis'
 'Androgenetic Alopecia' 'Angina' 'Angina Pectoris' 'Anorexia'
 'Anxiety Disorder' 'Aphthous Ulcers' 'Appendicitis' 'Arrhythmia' 'Asthma'
 "Athlete's Foot" 'Atopic Dermatitis' 'Atrial Fibrillation'
 'Attention Deficit Hyperactivity Disorder' 'Basal Cell Carcinoma'
 "Bell's Palsy" 'Benign Positional Vertigo' 'Bipolar Disorder'
 'Bronchial Asthma' 'Bronchitis' 'COVID-19' 'Carpal Tunnel Syndrome'
 'Cellulitis' 'Chronic Asthma' 'Chronic Bronchitis'
 'Chronic Fatigue Syndrome' 'Chronic Migraine'
 'Chronic Obstructive Pulmonary Disease'
 'Chronic Obstructive Pulmonary Disease (COPD)' 'Cluster Headaches'
 'Colorectal Cancer' 'Conjunctivitis' 'Constipation' 'Contact Dermatitis'
 "Cushing's Syndrome" 'Cyanosis' 'Cystic Acne' 'Cystitis'
 'Deep Vein Thrombosis' 'Depression' 'Diabetes' 'Diabetes Insipidus'
 'Diabetes Mellitus' 'Diabetes Type 1' 'Diabete

In [39]:
# Model archeitechture
input_layer = Input(shape=(max_length,))
embedding = Embedding(input_dim = 5000, output_dim = 64)(input_layer)
lstm_layer = LSTM(64)(embedding)

disease_output = Dense(len(label_encoder_disease.classes_), activation = 'softmax', name = 'disease_output')(lstm_layer)
prescription_output = Dense(len(label_encoder_prescription.classes_), activation = 'softmax', name = 'prescription_output')(lstm_layer)



KerasTensor(type_spec=TensorSpec(shape=(None, 178), dtype=tf.float32, name=None), name='disease_output/Softmax:0', description="created by layer 'disease_output'")


In [40]:
# compiling the model
model = Model(inputs = input_layer, outputs = [disease_output, prescription_output])

model.compile(loss = {'disease_output':'categorical_crossentropy', 'prescription_output':'categorical_crossentropy'}, optimizer = 'adam', metrics = {'disease_output':['accuracy'], 'prescription_output':['accuracy']})
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 17)]                 0         []                            
                                                                                                  
 embedding (Embedding)       (None, 17, 64)               320000    ['input_1[0][0]']             
                                                                                                  
 lstm (LSTM)                 (None, 64)                   33024     ['embedding[0][0]']           
                                                                                                  
 disease_output (Dense)      (None, 178)                  11570     ['lstm[0][0]']                
                                                                                             

In [41]:
# Training the model
model.fit(pad_sequence, {'disease_output': disease_labels_categorical,'prescription_output': prescription_labels_categorical}, epochs = 100, batch_size = 32)

Epoch 1/100


13/13 [==============================] - 3s 9ms/step - loss: 11.1449 - disease_output_loss: 5.1786 - prescription_output_loss: 5.9663 - disease_output_accuracy: 0.0025 - prescription_output_accuracy: 0.0025
Epoch 2/100
13/13 [==============================] - 0s 8ms/step - loss: 11.1237 - disease_output_loss: 5.1609 - prescription_output_loss: 5.9628 - disease_output_accuracy: 0.0344 - prescription_output_accuracy: 0.0025  
Epoch 3/100
13/13 [==============================] - 0s 8ms/step - loss: 11.0759 - disease_output_loss: 5.1109 - prescription_output_loss: 5.9650 - disease_output_accuracy: 0.0344 - prescription_output_accuracy: 0.0074
Epoch 4/100
13/13 [==============================] - 0s 9ms/step - loss: 10.9824 - disease_output_loss: 4.9771 - prescription_output_loss: 6.0053 - disease_output_accuracy: 0.0344 - prescription_output_accuracy: 0.0074
Epoch 5/100
13/13 [==============================] - 0s 17ms/step - loss: 10.8705 - disease_output_loss: 4.9055 - prescr

In [50]:
def make_prediction(prob_statement):
    # converting the input into padded sequence
#     tokenizer = Tokenizer(num_words = 5000, oov_token = "<OOV>")
#     tokenizer.fit_on_texts([prob_statement])
    # preprocessing the input
    sequence = tokenizer.texts_to_sequences([prob_statement])
    pad = pad_sequences(sequence, maxlen = max_length, padding = 'post')
    prediction = model.predict(pad)
    
    #decoding the output
    disease_index = np.argmax(prediction[0], axis=1)[0]
    prescription_index = np.argmax(prediction[1], axis=1)[0]
    
    
    disease_predicted = label_encoder_disease.inverse_transform([disease_index])[0]
    prescription_predicted = label_encoder_prescription.inverse_transform([prescription_index])[0]
    
    
    print(f"Predicted Disease: {disease_predicted}")
    print(f"Suggested Prescription: {prescription_predicted}")
    
#     print(output)

patient_input = "I've experienced a loss of appetite and don't enjoy food anymore."
make_prediction(patient_input)

1/1 [==============================] - 0s 13ms/step
Predicted Disease: Depression
Suggested Prescription: Antidepressants; eating nutrient-rich foods.
